In [1]:
import os

import pandas as pd

import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt

%matplotlib inline

#load the dataframes
df_train  = pd.read_csv('~/git/homeprices_kaggle/train.csv', parse_dates=True)
df_test = pd.read_csv('~/git/homeprices_kaggle/test.csv',  parse_dates=True)



# Helper Functions

In [2]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())


In [ ]:
def dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked', 'Ticket_Lett', 'Cabin_Letter', 'Name_Title', 'Fam_Size']):
    for column in columns:
        train[column] = train[column].apply(lambda x: str(x))
        test[column] = test[column].apply(lambda x: str(x))
        good_cols = [column+'_'+i for i in train[column].unique() if i in test[column].unique()]
        train = pd.concat((train, pd.get_dummies(train[column], prefix = column)[good_cols]), axis = 1)
        test = pd.concat((test, pd.get_dummies(test[column], prefix = column)[good_cols]), axis = 1)
        del train[column]
        del test[column]
    return train, test

# EDA

In [3]:
# Ok, so our goal is to predict the sale price of the home in the test dataset.  

# First, lets take a look at our columns in the dataframe to see if we have any nulls to deal with
df_train.columns[df_train.isnull().any()]
#temp_ar = df_train.columns[df_train.isnull().any()]

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

In [4]:
# QUESTION:  ok, quite a few of them.  How many nulls in each column?
# SOLUTION:  Lets put all the columns with nulls into a subset to analyze further

missingHousingData = df_train[['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature']].copy()


In [5]:
#what does that missingHousingData dataframe look like?
missingHousingData.head()

,LotFrontage,Alley,MasVnrType,MasVnrArea,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature
0,65.0,NaN,BrkFace,196.0,Gd,TA,No,GLQ,Unf,SBrkr,NaN,Attchd,2003.0,RFn,TA,TA,NaN,NaN,NaN
1,80.0,NaN,None,0.0,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Attchd,1976.0,RFn,TA,TA,NaN,NaN,NaN
2,68.0,NaN,BrkFace,162.0,Gd,TA,Mn,GLQ,Unf,SBrkr,TA,Attchd,2001.0,RFn,TA,TA,NaN,NaN,NaN
3,60.0,NaN,None,0.0,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Detchd,1998.0,Unf,TA,TA,NaN,NaN,NaN
4,84.0,NaN,BrkFace,350.0,Gd,TA,Av,GLQ,Unf,SBrkr,TA,Attchd,2000.0,RFn,TA,TA,NaN,NaN,NaN


# Examing the missing data -- MiscFeature and corresponding MiscVal columns.

In [6]:
#how many for each column?
missingHousingData.isnull().sum()

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [7]:
#quite a few more missing from some columns vs others, what about % missing?
missingHousingData.isnull().sum()/len(missingHousingData)

LotFrontage     0.177397
Alley           0.937671
MasVnrType      0.005479
MasVnrArea      0.005479
BsmtQual        0.025342
BsmtCond        0.025342
BsmtExposure    0.026027
BsmtFinType1    0.025342
BsmtFinType2    0.026027
Electrical      0.000685
FireplaceQu     0.472603
GarageType      0.055479
GarageYrBlt     0.055479
GarageFinish    0.055479
GarageQual      0.055479
GarageCond      0.055479
PoolQC          0.995205
Fence           0.807534
MiscFeature     0.963014
dtype: float64

In [8]:
# I'm seeing columns with more than 25% of data missing, lets see whats in those before we drop them
# MiscFeature could be so random and unique per row if its like 'english tea parlor room'

In [9]:
#show me all the unique values for MiscFeature
missingHousingData.MiscFeature.unique()

array([nan, 'Shed', 'Gar2', 'Othr', 'TenC'], dtype=object)

In [10]:
# are they all just one-offs?  No, Shed has 49 occurences, a decent amount.  
# I don't think I want to drop this column, as it being populated probably ups the sale price
missingHousingData['MiscFeature'].value_counts()

Shed    49
Othr     2
Gar2     2
TenC     1
Name: MiscFeature, dtype: int64

In [11]:
# From reviewing the data_description.txt file, we know there is a column in the original 
# training set called MiscVal that is the #value of the MiscFeature item.  
#Those have to impact sale price,

# so i think we just want to fill in the na values with a 'NA' string for now, and drop it from our 
# missingHousingData dataframe.
missingHousingData.drop(['MiscFeature'], axis=1, inplace=True)
#df_train['MiscFeature'].fillna('NA', inplace=True)
missingHousingData.head()

,LotFrontage,Alley,MasVnrType,MasVnrArea,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageQual,GarageCond,PoolQC,Fence
0,65.0,NaN,BrkFace,196.0,Gd,TA,No,GLQ,Unf,SBrkr,NaN,Attchd,2003.0,RFn,TA,TA,NaN,NaN
1,80.0,NaN,None,0.0,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Attchd,1976.0,RFn,TA,TA,NaN,NaN
2,68.0,NaN,BrkFace,162.0,Gd,TA,Mn,GLQ,Unf,SBrkr,TA,Attchd,2001.0,RFn,TA,TA,NaN,NaN
3,60.0,NaN,None,0.0,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Detchd,1998.0,Unf,TA,TA,NaN,NaN
4,84.0,NaN,BrkFace,350.0,Gd,TA,Av,GLQ,Unf,SBrkr,TA,Attchd,2000.0,RFn,TA,TA,NaN,NaN


In [12]:
# Ok, regarding that MiscVal column -- sounds like the MiscVal column contains the $ value
# of that column.  So why keep MiscFeature?  We have the $ amount in MiscVal, which probably
# bears more precision in terms of impact to sale amount.  Here is were I would A|B test the model to 
# see if any impact



# Execute Helper functions / transform data

## A|B Test 1 - MiscFeature removal

In [13]:
# doubt this column matters
df_train.drop('MiscFeature', axis=1, inplace=True)

In [19]:
#some of these missing columns we'll insert 'N' for the value, eventually that
#will get digitized.

df_train.isnull().sum()/len(df_train)

Id               0.000000
MSSubClass       0.000000
MSZoning         0.000000
LotFrontage      0.177397
LotArea          0.000000
Street           0.000000
Alley            0.000000
LotShape         0.000000
LandContour      0.000000
Utilities        0.000000
LotConfig        0.000000
LandSlope        0.000000
Neighborhood     0.000000
Condition1       0.000000
Condition2       0.000000
BldgType         0.000000
HouseStyle       0.000000
OverallQual      0.000000
OverallCond      0.000000
YearBuilt        0.000000
YearRemodAdd     0.000000
RoofStyle        0.000000
RoofMatl         0.000000
Exterior1st      0.000000
Exterior2nd      0.000000
MasVnrType       0.000000
MasVnrArea       0.005479
ExterQual        0.000000
ExterCond        0.000000
Foundation       0.000000
                   ...   
HalfBath         0.000000
BedroomAbvGr     0.000000
KitchenAbvGr     0.000000
KitchenQual      0.000000
TotRmsAbvGrd     0.000000
Functional       0.000000
Fireplaces       0.000000
FireplaceQu 

In [15]:
# The 2 lines below, both alter a copy of the frame, which we don't want (inplace parm deceiving)
#df_train.BsmtCond.fillna('NA', inplace=True)
#df_train[['PoolQC', 'Fence', 'GarageCond', 'GarageQual', 'GarageFinish', 'GarageType', 'FireplaceQu', 'Electrical', 'BsmtFinType2', 'BsmtFinType1', 'BsmtExposure', 'BsmtCond', 'BsmtQual', 'MasVnrType', 'Alley']].fillna('NA', inplace=True)

#Not pretty, but does the job  - so we should not have any nanValues that are string type
df_train.fillna({'PoolQC': 'NA', 'Fence': 'NA', 'GarageCond': 'NA', 'GarageQual': 'NA', 'GarageFinish': 'NA', 'GarageType': 'NA', 'FireplaceQu': 'NA', 'Electrical': 'NA', 'BsmtFinType2': 'NA', 'BsmtFinType1': 'NA', 'BsmtExposure': 'NA', 'BsmtCond': 'NA', 'BsmtQual': 'NA', 'MasVnrType': 'NA', 'Alley': 'NA'}, inplace=True)

In [20]:
#validate one of the columns to make sure no more nan
df_train.MasVnrArea.unique()

array([1.960e+02, 0.000e+00, 1.620e+02, 3.500e+02, 1.860e+02, 2.400e+02,
       2.860e+02, 3.060e+02, 2.120e+02, 1.800e+02, 3.800e+02, 2.810e+02,
       6.400e+02, 2.000e+02, 2.460e+02, 1.320e+02, 6.500e+02, 1.010e+02,
       4.120e+02, 2.720e+02, 4.560e+02, 1.031e+03, 1.780e+02, 5.730e+02,
       3.440e+02, 2.870e+02, 1.670e+02, 1.115e+03, 4.000e+01, 1.040e+02,
       5.760e+02, 4.430e+02, 4.680e+02, 6.600e+01, 2.200e+01, 2.840e+02,
       7.600e+01, 2.030e+02, 6.800e+01, 1.830e+02, 4.800e+01, 2.800e+01,
       3.360e+02, 6.000e+02, 7.680e+02, 4.800e+02, 2.200e+02, 1.840e+02,
       1.129e+03, 1.160e+02, 1.350e+02, 2.660e+02, 8.500e+01, 3.090e+02,
       1.360e+02, 2.880e+02, 7.000e+01, 3.200e+02, 5.000e+01, 1.200e+02,
       4.360e+02, 2.520e+02, 8.400e+01, 6.640e+02, 2.260e+02, 3.000e+02,
       6.530e+02, 1.120e+02, 4.910e+02, 2.680e+02, 7.480e+02, 9.800e+01,
       2.750e+02, 1.380e+02, 2.050e+02, 2.620e+02, 1.280e+02, 2.600e+02,
       1.530e+02, 6.400e+01, 3.120e+02, 1.600e+01, 

In [21]:
#lets see some of these numeric column null values in action
df_train.LotFrontage.unique()

array([ 65.,  80.,  68.,  60.,  84.,  85.,  75.,  nan,  51.,  50.,  70.,
        91.,  72.,  66., 101.,  57.,  44., 110.,  98.,  47., 108., 112.,
        74., 115.,  61.,  48.,  33.,  52., 100.,  24.,  89.,  63.,  76.,
        81.,  95.,  69.,  21.,  32.,  78., 121., 122.,  40., 105.,  73.,
        77.,  64.,  94.,  34.,  90.,  55.,  88.,  82.,  71., 120., 107.,
        92., 134.,  62.,  86., 141.,  97.,  54.,  41.,  79., 174.,  99.,
        67.,  83.,  43., 103.,  93.,  30., 129., 140.,  35.,  37., 118.,
        87., 116., 150., 111.,  49.,  96.,  59.,  36.,  56., 102.,  58.,
        38., 109., 130.,  53., 137.,  45., 106., 104.,  42.,  39., 144.,
       114., 128., 149., 313., 168., 182., 138., 160., 152., 124., 153.,
        46.])

In [22]:
#Now lets take care of the numeric column null values -- 
# put the mean or median for each column in that columns nan values
# starting with the year garage was built, 

df_train['GarageYrBlt'].fillna((df_train['GarageYrBlt'].median()), inplace=True)
df_train['MasVnrArea'].fillna((df_train['MasVnrArea'].mean()), inplace=True)
df_train['LotFrontage'].fillna((df_train['LotFrontage'].mean()), inplace=True)


#verify
df_train.GarageYrBlt.unique()

array([2003., 1976., 2001., 1998., 2000., 1993., 2004., 1973., 1931.,
       1939., 1965., 2005., 1962., 2006., 1960., 1991., 1970., 1967.,
       1958., 1930., 2002., 1968., 2007., 2008., 1957., 1920., 1966.,
       1959., 1995., 1954., 1953., 1980., 1983., 1977., 1997., 1985.,
       1963., 1981., 1964., 1999., 1935., 1990., 1945., 1987., 1989.,
       1915., 1956., 1948., 1974., 2009., 1950., 1961., 1921., 1900.,
       1979., 1951., 1969., 1936., 1975., 1971., 1923., 1984., 1926.,
       1955., 1986., 1988., 1916., 1932., 1972., 1918., 1924., 1996.,
       1940., 1949., 1994., 1910., 1978., 1982., 1992., 1925., 1941.,
       2010., 1927., 1947., 1937., 1942., 1938., 1952., 1928., 1922.,
       1934., 1906., 1914., 1946., 1908., 1929., 1933.])

In [25]:
df_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non

# A-B Test 2 --drop_first parameter on get_dummies ?

In [32]:
#convert columns to categorical
df_train2 = pd.get_dummies(df_train, drop_first=True)

df_train2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 259 entries, Id to SaleCondition_Partial
dtypes: float64(3), int64(35), uint8(221)
memory usage: 748.6 KB


In [35]:
print(df_train2)

        Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  \
0        1          60    65.000000     8450            7            5   
1        2          20    80.000000     9600            6            8   
2        3          60    68.000000    11250            7            5   
3        4          70    60.000000     9550            7            5   
4        5          60    84.000000    14260            8            5   
5        6          50    85.000000    14115            5            5   
6        7          20    75.000000    10084            8            5   
7        8          60    70.049958    10382            7            6   
8        9          50    51.000000     6120            7            5   
9       10         190    50.000000     7420            5            6   
10      11          20    70.000000    11200            5            5   
11      12          60    85.000000    11924            9            5   
12      13          20    70.049958   

# Hyperparameter and rf or regression execution

In [38]:


from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_features='auto', oob_score=True, random_state=1, n_jobs=-1)

param_grid = { "criterion" : ["gini", "entropy"], "min_samples_leaf" : [1, 5, 10], "min_samples_split" : [2, 4, 10, 12, 16], "n_estimators": [50, 100, 400, 700, 1000]}

gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)

gs = gs.fit(df_train2.iloc[:, 1:], df_train2.iloc[:, 0])

print(gs.bestscore)
print(gs.bestparams)
print(gs.cvresults)


ValueError: n_splits=3 cannot be greater than the number of members in each class.

# Post Model analysis

# submission file generation